## verzeichnisse_extract_data.ipynb

In this notebook, we will extract the text data enclosed in the Verzeichnisse. The data has been pre-OCR'ed, with the layout-ordered text present in the pdf. We will use an algorithm to extract this layout-specific text to order the data in rows and column. 

In [31]:
import re
import os
import fitz
import glob
import json
import ollama
import pdfplumber
from fpdf import FPDF

# OpenAI
from openai import OpenAI
api_key = os.getenv('OPENAI_API_KEY')

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

client = OpenAI()

ollama.pull("llama3.1")
ollama.pull("philsaysss/german-language")

# Function to crop the .pdf
def crop_pdf(pdf_path, output_path, x1=0, x2=180):
    doc = fitz.open(pdf_path)
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        rect = page.mediabox
        crop_rect = fitz.Rect(x1, rect.y0, x2, rect.y1)
        page.set_cropbox(crop_rect)
        
    doc.save(output_path)
    doc.close()    
    
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        all_text = ""
        
        # Loop through all the pages in the PDF
        for page in pdf.pages:
            # Extract text from each page
            text = page.extract_text()
            if text:
                all_text += text
        
    return all_text

def split_text_by_pattern(text):
    # Regex pattern to match a digit followed by a space
    pattern = r'\n\d+\s'
    
    # Split the text using the regex pattern
    split_list = re.split(pattern, text)
    
    return split_list

def crop_pdf_and_extract_text(pdf_path, output_path, x1=0, y1=0, x2=180, y2=722):
    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        # Initialize the text container
        extracted_text = ""
        
        # Create a new PDF with fpdf to store cropped text
        pdf_writer = FPDF()
        
        # Iterate over all the pages in the PDF
        for page_num in range(len(pdf.pages)):
            page = pdf.pages[page_num]
            
            # Define the cropping box (x1, y1, x2, y2)
            crop_box = (x1, y1, x2, y2)
            
            # Crop the page to the defined box
            cropped_page = page.within_bbox(crop_box)
            
            # Extract the text from the cropped area
            cropped_text = cropped_page.extract_text()
            
            # Add the cropped text to the total extracted text
            if cropped_text:
                cropped_text = cropped_text.encode('latin-1', 'replace').decode('latin-1')
                extracted_text += cropped_text + "\n"
                
                # Add a new page to the new PDF document
                pdf_writer.add_page()
                pdf_writer.set_xy(10, 10)
                pdf_writer.set_font("Arial", size=12)
                pdf_writer.multi_cell(0, 10, cropped_text)
        
        # Save the new PDF with cropped text
        pdf_writer.output(output_path)

        # Optionally, return the extracted text
        return extracted_text

def request_city_llm(extracted_text):
    responses = []
    for text in extracted_text:
        prompt = f"""
        Welche Stadt wird in diesem Text genannt - der Städtename kann durch einige Füllwörter getrennt werden. 
        Gib mir nur den Städtename, nichts anderes: {text}
        """
        response = ollama.chat(model='philsaysss/german-language', messages=[
            {
                'role': 'user',
                'content': prompt,
                },
            ],
                               options={'temperature':0})
        
        output = response['message']['content']
        responses.append(output)
    return responses


def request_city_gpt(extracted_text):
    responses = []
    for text in extracted_text:
        prompt = f"""
        Welche Stadt wird in diesem Text genannt? Der Städtename kann durch einige Füllwörter getrennt sein. Gib mir nur den Städtenamen, nichts anderes: {text}
        """
        message = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
        ]
        response = client.chat.completions.create(
        model="gpt-4o",
        messages=message,
        max_tokens=2500
        )
        responses.append(response)
        
    return responses
        

## Cropping Pipeline

- We crop the file, then extract the text from the cropped file. Then we use a LLM pipeline to discover the city mentioned. 

In [7]:
# Example usage:
pdf_file_path = '../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1853_A.pdf'
output_path = '../../data/patent_data/raw_patent_data/Verzeichnisse/cropped_output.pdf'
extracted_text = crop_pdf_and_extract_text(pdf_file_path, output_path, x1=0, y1=0, x2=180, y2=722)

print(extracted_text)
splitted_text = split_text_by_pattern(extracted_text)
print(splitted_text)

Fortlaufende
Vor- und Zuname, Charakter
Bahl
und
Wohnort des Privilegirten
1 Adensamer Joseph, lan Ver
desbefugter Bandfabrikant Be
in Wien ( Schottenfeld) . in
Au
he
2 Adler Carl , Fabrikant Ver
chemisch. Producte in Wien te
( Gumpendorf Nr. 190) . un
bin
ku
Fe
Er
miadar Ar
ste
3 Alcan Michael , Civil -I n Erfi
genieur, und Peter Hypolit ein
Limet in Paris ( Bevoll Se
mächtigter Jacob Franz Ab
Heinrich Hemberger, Fri
Verwaltungs- Director in bar
Wien) . Tig
4 Allan Thomas , Ingenieur Erfi
in Edinburg , durch sei tris
nen Bevollmächtigten Georg
Märkl, in Wien (J oseph
stadt Nr. 65 ) .ad
5 Altlechner Laurenz , Bür Erfi
ger in Wien (L aimgrube un
Nr. 182) .
miadar
6 Amstötter Mathias , und Erfi
Johann Schredl , in Wie vo
ner Neustadt Nr . 172. chi
7 Anthon Ernst Friedrich , Erfi
technisch . Chemiker in Prag, zur
Nr. 70 - IV . höc
mistop Ve
sec
int zu
KaC
8 Armengaud Jacob Eugen V
sen. , Civil Ingenieur in sch
Paris (R ue St. Sebastien zus
Nr. 45) , durch Jacob Franz Er
Heinrich Hemberger

In [4]:
# For loop implementation
files = glob.glob(os.path.join("../../data/patent_data/raw_patent_data/Verzeichnisse/", "Verzeichniss*"))
verzeichnisse_texts = []
for file in files:
    print(file)
    output_path = '../../data/patent_data/raw_patent_data/Verzeichnisse/cropped_output.pdf'
    extracted_text = crop_pdf_and_extract_text(file, output_path, x1=0, y1=0, x2=180, y2=722)
    splitted_text = split_text_by_pattern(extracted_text)
    verzeichnisse_texts.append(splitted_text)
    

../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1855_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1854_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1859_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1856_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1858_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1863_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1864_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1853_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1860_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1857_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1867_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1865_A.pdf
../../data/patent_data/raw_patent_data/Verzeichnisse/Verzeichniss_1862_A.pdf

In [12]:
# Flatten the resulting lists
verzeichnisse_texts_fl = [item for sub_list in verzeichnisse_texts for item in sub_list]

In [23]:
# Create a .json file to send them to GPT Batch API
list_of_dictionaries = []

for num, text in enumerate(verzeichnisse_texts_fl):
    # Set up the dictionary for each text
    batch_dictionary = {}
    batch_dictionary['custom_id'] = f"request-{num}"
    batch_dictionary['method'] = "POST"
    batch_dictionary['url'] = "/v1/chat/completions"
    # Create the prompt we want GPT to answer
    prompt = f"Welche Stadt wird in diesem Text genannt? Der Städtename kann durch einige Füllwörter getrennt sein. Gib mir nur den Städtenamen, nichts anderes. Falls es keinen Namen gibt, gib mir 'NA' zurück: {text}"
    batch_dictionary['body'] = {"model": "gpt-4o", "messages": [{"role": "user", "content": prompt}], "max_tokens": 100}
    list_of_dictionaries.append(batch_dictionary)

In [37]:
# With this list of dictionaries, we create a jsonlite file and then a GPT Batch Object:
with open("../../data/batch_verzeichnisse.json", "w") as outfile: 
    for i in list_of_dictionaries:
        json.dump(i, outfile)
        outfile.write("\n")

## A couple of alternatives to check out

- https://ollama.com/marco/em_german_mistral_v01
- https://colab.research.google.com/drive/1IJfJdVwGkfe5MYOqHptystR3FBeEUdGn?usp=sharing
- https://github.com/jphme/EM_German?tab=readme-ov-file
- 